## EDGE Scores and Conservation Strategy

### 1. Introduction and resources

This practical is aimed to introduce you to the EDGE & FUDGE Scores that you'll need for your conservation strategy coursework. ** Insert info on EDGE Scores **

https://www.zsl.org/conservation/our-priorities/wildlife-back-from-the-brink/animals-on-the-edge 

### 2. Preparing your Data

To calculate EDGE metrics, we need data on the species we're interested in, and their phylogenetic relationship. For the coursework we're interested in a species list from a specific location or clade. This may be a country, a national park, or a clade such as family or order.

To do this we will use information from the BirdLife International data zone. 

http://datazone.birdlife.org/home

You can search for specific locations by site (called IBA) or country options. For this practical we will be using data on birds found in the United Kingdom. **Pick a different country for your coursework!** Once you've found your country, click on the link for the TOTAL number of species (267 in the image below), and you'll be redirected to a page where you can download your species list as a csv file. You'll need to supply an email at the next stage.

**Don't use your Imperial email. In the past these emails have been significantly delayed for some reason, and you will not be able to access your species list for a long time!!! Use a personal email such as gmail instead.**

![image.png](attachment:image.png)

Once you've downloaded your species list, make sure it's saved in the correct directory. The majority of issues with reading in data is caused by not being in the correct directory, or spelling errors in the file pathway. Please also rename your csv file to something sensible to avoid typos. Now we'll read in the data and check it, which you should be getting used to.

In [ ]:
dir.create("My Git Repo")
git2r::clone("https://github.com/Syrph/BCB_Practicals", "My Git Repo")
setwd("My Git Repo")
source("install.R")

In [ ]:
UK_birds <- read.csv("uk_bird_species.csv", header = T)
head(UK_birds)
str(UK_birds)

We've got our species list, but it's using the birdlife taxonomy. For this practical we'll be using a phylogenetic tree based on the Jetz taxonomy. For more info on the tree, and where download your own in the future, look here:

http://birdtree.org/

So we have to convert our Birdlife names to Jetz names. Because the Birdlife taxonomy is newer, there may be name changes or species splits in our species list. We'll use a crosswalk from the Tobias Lab to convert our names to Jetz, and check for any species that don't neatly fit in. 



In [ ]:
# Read in our crosswalk.
crosswalk <- read.csv("crosswalk.csv")
head(crosswalk)

# Load dplyr for the join functions.
library(dplyr)

# Change the column name so that birdlife names match on the UK_birds and crosswalk dataframes. This is what we'll match up by.
colnames(UK_birds)[2] <- "Birdlife_Name"

# Left join is a function that will join two dataframes. It takes the left arguement (UK_birds), and trys to join up matching rows from the crosswalk. 
# By specifying left, we're saying we don't want any unmatched rows from the crosswalk, but we'll keep unmatched rows from UK_birds. Use ? for more info.
UK_Jetz <- left_join(UK_birds, crosswalk)

Now we've joined the two species lists we need to check for duplicates due to splits, and any species we could match up. Speices that didn't match will have `NA` for the Jetz_Name.

In [ ]:
# Pulls out all the rows with NA in the Jetz Name
UK_Jetz[is.na(UK_Jetz$Jetz_Name),]
nrow(UK_Jetz)

We can see from the IUCN category that the Great Auk is now extinct so we'll remove it from our dataframe using another NA removal function from the `tidyr` package.

In [ ]:
# Load in tidyr
library(tidyr)

# If you just want to remove NAs from a specific column, the drop NA function does this nicely.
UK_Jetz <- UK_Jetz %>% drop_na(Jetz_Name)

Great. Now that's all the species that we couldn't match. But there's also might be duplicates, where we have multiple birdlife species to only one Jetz species. This happens when a Jetz species has been split after the tree was made. There may also be some lumps where mutliple Jetz species have one Birdlife Name. We can easily find them using a convient function from the `Janitor` package.

In [ ]:
install.packages("janitor")
library(janitor)

# The get_dupes function allows us to specify which column we want to check for duplicates in.
lumps <- UK_Jetz %>% get_dupes(Birdlife_Name)
splits <- UK_Jetz %>% get_dupes(Jetz_Name)

So there's no splits or lumps for the UK species, but check your species list as well. If there are splits, open your dataframe in excel and search for those species. Species tend to be split into parent species, with the same *species* name, and daughter species, normally with a new name. The genus name might have changed as well, and sometimes the spelling changes slightly so check for all these things! We want the parent species so keep that one. If you do have splits, your new species list will be slightly smaller.

Now we have our list of species ready to go. We also need a tree to use. We'll load the phylogenetic packages and read in our tree.

In [ ]:
library(ape)
library(caper)

# Load in and plot the tree
bird_tree <- read.tree("all_birds.tre")
plot(bird_tree)

### 2. ED Scores

Now that we've got our tree and our species we can start calculating our ED (Evolutionary Distinctiveness) scores. Because we are calculating the evolutionary distinctiveness of our UK species, we want to use the whole bird phylogeny to compare against. Then we can find then out if our species in the UK are very closely related to others in the tree, or represent distinct lineages that might want to conserve to protect valuable evolutionary diversity.

We can do this easily using a simple function from the `caper` package. This sometimes takes a while to run.

In [ ]:
# We can first transform our tree into a matrix of distances from each tip to tip. This step is optional but stops a warning message from ed.calc, which prefers a matrix to a tree.
bird_matrix <- clade.matrix(bird_tree)

# Now we can run the ed.calc function, which calculates ED scores for each species. The output gives two dataframes, but we only want the species names and scores so we use $spp
ED <- ed.calc(bird_matrix)$spp
head(ED)

Now that we've got our ED scores for each species, we need to log transform and normalise our scores. 

In [ ]:
# By adding 1 to our scores, this prevents negative logs when our ED scores are below 1. 
ED$EDlog <- log(1+ED$ED)

# We can normalise our scores so they're scaled between 0 and 1
ED$EDn <- (ED$EDlog - min(ED$EDlog)) / (max(ED$EDlog) - min(ED$EDlog))
head(ED)

Now that we have our normalised scores for all birds, we need to subset the list for just UK species.

In [ ]:
# Pull out the ED row numbers for our species list.
row_numbers <- (ED$species %in% UK_Jetz$Jetz_Name)

# Get our UK species with ED scores
UK_ED <- ED[row_numbers,]
str(UK_ED)

We now have the ED scores of 266 species found in the UK. With these scores we can see how unique our species are in terms of the evolutionary pathway.

In [ ]:
# Find the highest ED score
UK_ED[UK_ED$EDn == max(UK_ED$EDn),]

The highest ED score belongs to Gavia stellata, the Red-throated diver. This species is part of the order Gaviiformes, which only includes five species in one genus. Therefore, with so few close relatives, we might consider this species a conservation priority to protect as much diversity as we can. However we don't yet know if this species needs conserving...

## 3. EDGE Scores

This is where EDGE scores come in. By combining ED scores with IUCN categories we can select the species that need conservation action, and represent unique evolutionary variation.

First we need to convert the IUCN status in GE (Globally Endangered) scores. This is relatively simple as we're just assigning numeric rankings. Because there's only a few different categories, we'll change them manually.

In [ ]:
# Create an empty column to store our GE scores.
UK_Jetz$GE <- NA

# Change the GE scores to the correct number depending on the ranking.
UK_Jetz[UK_Jetz$Global.IUCN.Red.List.Category == "LC", "GE"] <- 0
UK_Jetz[UK_Jetz$Global.IUCN.Red.List.Category == "NT", "GE"] <- 1
UK_Jetz[UK_Jetz$Global.IUCN.Red.List.Category == "DD", "GE"] <- 1
UK_Jetz[UK_Jetz$Global.IUCN.Red.List.Category == "VU", "GE"] <- 2
UK_Jetz[UK_Jetz$Global.IUCN.Red.List.Category == "EN", "GE"] <- 3
UK_Jetz[UK_Jetz$Global.IUCN.Red.List.Category == "CR", "GE"] <- 4

Now we'll merge our GE scores with our ED scores in one dataframe.

In [ ]:
# Join the last two columns of UK_Jetz to ED scores. This time we'll use the by arguement rather than change the column names.
EDGE <- left_join(UK_ED, UK_Jetz[,6:7], by = c("species" = "Jetz_Name"))
head(EDGE)

We can now calculate our EDGE scores using some simple maths:

$$EDGE=ln⁡(1+ED)+GE×ln⁡(2)$$

We have already done the first half. Now we just need to multiply GE scores by the natural log of 2, and combine them.

In [ ]:
# The log function uses natural logarithims by default.
EDGE$EDGE <- EDGE$EDlog + EDGE$GE * log(2)

head(EDGE)

Now we have our EDGE scores, we can see if our conservation priority has changed in light of IUCN categories.

In [ ]:
# Find the highest EDGE score
EDGE[EDGE$EDGE == max(EDGE$EDGE),]

# Find the EDGE score for Gavia stellata
EDGE[EDGE$species == "Gavia_stellata",]

So now we can see that the top conservation priority is Puffinus mauretanicus, the Balearic shearwater. Whilst G. stellata is still high, it's low IUCN score means its less of a priority than P. maurentanicus, which is critically endangered. 

In reality, you might include both species in a conservation plan for different reasons. We can see from the spread of EDGE scores that there are few species above 3, and we would ideally like to create a plan that maximises the conservation of all of them (if it's possible).

In [ ]:
hist(EDGE$EDGE, breaks = 20)

In [ ]:
# With the filter function we can split our dataframes based on rules for certain columns.
EDGE %>% filter(EDGE > 3)

## 4. FUDGE Scores

Instead of evolutionary distinctiveness, we might instead be interested in what functional traits each species provides. Species with low functional diversity may be 'functionally redundant' in the ecosystem, whereas those with high functional diversity may provide key ecosystem services that aren't easily replaceable. 

Unlike ED, we will not calculate functional distinctiveness (FD and FDn) in relation to all species within the order worldwide. Instead, we will calculate FD and FDn for just our chosen species. The reason for this is that FD is traditionally used in the context of a specific community or radiation of species (i.e. all birds found within a national park, or all species of lemur).

First we'll load in some bird trait data of body mass and beak length.

In [ ]:
bird_traits <- read.csv("bird_traits.csv")
head(bird_traits)
str(bird_traits)

Now we just want the species from the UK.

In [ ]:
# Pull out the ED row numbers for our species list.
row_numbers <- (bird_traits$Jetz_Name %in% UK_Jetz$Jetz_Name)

# Get our UK species with ED scores
UK_traits <- bird_traits[row_numbers,]
str(UK_traits)

We next need to change row names to species names and remove the species column. Then normalise our trait data so that body_mass and beak have the same scale (the same variance). 


In [ ]:
rownames(UK_traits) <- UK_traits$Jetz_Name
UK_traits$Jetz_Name <- NULL

# Make each column have the same scale.
UK_traits <- scale(UK_traits, scale=T)
head(UK_traits)

To calculate functional diversity we'll create a distance matrix of our traits. Species with similar traits will have smaller 'distances'.

In [ ]:
# Create a matrix
traits_matrix <- as.matrix(UK_traits)

# Converts traits into 'distance' in trait space.
distance_matrix <- dist(traits_matrix)

The next step is to create a new tree using the neighbour-joining method (Saitou & Nei, 1987) (Google for more information!). This will create a tree where branch lengths show how similar species are in trait space rather than evolutionary distance. This function may take a while with more species so don't be alarmed if the group you've chosen takes much longer.

In [ ]:
# Create the tree
trait_tree <- nj(distance_matrix)

# Test to see if it's worked. The tree looks different to a normal one because tips don't line up neatly at the present time period like with evolutionary relationships.
plot(trait_tree, cex=0.4)

FD trees can fail if there are too many NAs in the data. If this is the case for your taxa, either impute missing data using genus averages (following Swenson et al. 2013) or remove species or traits with high NA counts from FD analysis. Note, however, that the bird data is very complete so there should be no need to remove NA species from the dataset; this should be a last resort so only do this if the analyses are failing repeatedly.  

With our tree of functional space, we can now calculate FD scores the same way we calculated ED scores. 

In [ ]:
# Create a matrix of distance from tip to tip.
tree_matrix <- clade.matrix(trait_tree)

# Calculate FD scores.
FD <- ed.calc(tree_matrix)$spp

# Change the name to FD
colnames(FD)[2] <- "FD"
head(FD)

Log and normalise the data as we did before with ED so we could compare FD scores from different groups. 

In [ ]:
FD$FDlog <- log(1+FD$FD)
FD$FDn <- (FD$FDlog - min(FD$FDlog)) / (max(FD$FDlog) - min(FD$FDlog))

# Find the highest FD score
FD[FD$FDn == max(FD$FDn),]

So the species with the largest FD score is the Cygnus olor, the mute swan. Not suprising seeing as mute swans are one of the heaviest flying birds alive today! We can also combine GE scores to see how IUCN categories change our priorities. We use the same formula as before:

$$FUDGE=ln⁡(1+FD)+GE×ln⁡(2)$$

In [ ]:
# Join FD and GE scores
FUDGE <- left_join(FD, UK_Jetz[,6:7], by = c("species" = "Jetz_Name"))

# Calculate FUDGE scores
FUDGE$FUDGE <- FUDGE$FDlog + FUDGE$GE * log(2)
head(FUDGE)

And does IUCN categories change our conservation priorities?

In [ ]:
# Find the highest EDGE score
FUDGE[FUDGE$FUDGE == max(FUDGE$FUDGE),]

# Find the EDGE score for Cygnus olor
FUDGE[FUDGE$species == "Cygnus_olor",]

Yes! Funnily enough the Balearic shearwater is again the species we need to check. This is because the GE component of FUDGE scores is  weighted much higher than the FD component. In fact, looking at FDn, the Balearic shearwater has a much lower score. This is one of the criticisms of FUDGE scores, and you may wish to alter your GE scores if you want a more even contribution from both parts. 

CHECK THIS!!! 

In [ ]:
hist(FUDGE$FDn, breaks = 20)

## 5. EcoEDGE Scores

So we've used EDGE scores to combine extinction risk with evolutionary diversity, and FUDGE scores to do the same with functional diversity. However, both are important, and we might want to combine all three into one metric. This is exactly what EcoEDGE scores do. And we've pretty much done all the hard work already. The equation is similar to the ones we've used, but we give ED and FD scores equal weighting:

$$EcoEDGE= (0.5×EDn + 0.5×FDn) + GE×ln⁡(2)$$

And remember our EDn and FDn scores have already been logged, so we don't need to log them now.

In [ ]:
# Merge FD and ED scores.
EcoEDGE <- left_join(EDGE, FUDGE)

# Calculate EcoEDGE scores
EcoEDGE$EcoEDGE <- (0.5*EcoEDGE$EDn + 0.5*EcoEDGE$FDn) + EcoEDGE$GE*log(2)
head(EcoEDGE)

We can again look at the spread and see which are the highest species.

In [ ]:
# Get the highest scoring species
EcoEDGE[EcoEDGE$EcoEDGE == max(EcoEDGE$EcoEDGE),]

# Get the top 10% of EcoEDGE scores.
EcoEDGE[EcoEDGE$EcoEDGE > quantile(EcoEDGE$EcoEDGE, 0.9),]

# See the spread
hist(EcoEDGE$EcoEDGE, breaks = 20)

Unsuprisingly, the Balearic shearwater is again the highest species. However, most birds in the UK are not currently threatened by extinction according to IUCN criteria. For your own country or taxa, this may be a very different story, and ED and FD scores will matter a lot more. How you chose to interpret and present your results is up to you, and will depend on the group that you've chosen.

For the practicals and coursework we've chosen to use a simplified version of EcoEDGE scores. If you're interested in learning more, check out this paper which first proposed the use of EcoEDGE scores:

https://onlinelibrary.wiley.com/doi/full/10.1111/ddi.12320

